In [28]:
import pandas as pd
from pathlib import Path
from datetime import timedelta


In [29]:
df = pd.read_csv("../data/raw_data.csv")

C:\Users\metal\AppData\Local\Temp\ipykernel_61756\3407229622.py:1: DtypeWarning: Columns (28,30,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/raw_data.csv")


In [30]:
def get_fx_date(launched_date):
    """Return the date to use for FX rate lookup.

    Mon–Fri: use the same day.
    Sat:     use Friday (one day earlier).
    Sun:     use Friday (two days earlier).
    """
    date = pd.to_datetime(launched_date)

    weekday = date.weekday() # 0=Mon ... 4=Fri, 5=Sat, 6=Sun

    if weekday == 5: # Saturday
        date = date - timedelta(days=1)
    elif weekday == 6: # Sunday
        date = date - timedelta(days=2)
        
    return date.normalize() 

In [31]:
def add_fx_mean(df: pd.DataFrame, fx_folder: str = "../data/fx_rates") -> pd.DataFrame:
    """Add fx_daily_mean for each row using daily FX CSVs and weekend logic.

    - launched_at is assumed to be epoch seconds.
    - For non-USD rows, FX rate is looked up from <currency>-USD_daily.csv
      using an FX date where Sat/Sun are mapped back to Friday.
    - For USD rows, fx_daily_mean is set to 1.0.

    Parameters
    ----------
    df : pd.DataFrame
        Project-level dataset with at least columns: 'currency', 'launched_at'.
    fx_folder : str, optional
        Folder path where the FX CSV files are stored.

    Returns
    -------
    pd.DataFrame
        Copy of the input DataFrame with new columns:
        - launched_date
        - fx_date
        - fx_daily_mean
    """
    
    df = df.copy()
    df["launched_date"] = pd.to_datetime(df["launched_at"], unit="s").dt.floor("D")
    
    # Luodaan fx_date: viikonloput -> perjantai
    df["fx_date"] = df["launched_date"].apply(get_fx_date)
    
    # Separate currencies to USD & others
    df_usd = df[df["currency"] == "USD"].copy()
    df_non_usd = df[df["currency"] != "USD"].copy()

    # USD - USD uses static 1.0 fx-rate
    df_usd["fx_daily_mean"] = 1.0
    
    # Get list of non-USD currencies
    currency_list = set(df_non_usd["currency"])

    merged_parts = []

    for currency in currency_list:
        df_sub = df_non_usd[df_non_usd["currency"] == currency].copy()

        # Get fx-rates from csv
        fx_path = f"{fx_folder}/{currency}-USD_daily.csv"
        fx = pd.read_csv(fx_path)

        fx["fx_date"] = pd.to_datetime(fx["timestamp"]).dt.floor("D")

        df_sub = df_sub.merge(
            fx[["fx_date", "fx_daily_mean"]],
            on = "fx_date",
            how = "left",
        )
        
        merged_parts.append(df_sub)

    df_final = pd.concat([df_usd] + merged_parts, ignore_index=True)

    return df_final 
    

In [32]:
df.head(5)

,collected_at,backers_count,blurb,category_id,category_name,category_parent_id,category_parent_name,category_url,converted_pledged_amount,country,...,profile_state_changed_at,project_url,spotlight,staff_pick,state,state_changed_at,static_usd_rate,usd_exchange_rate,usd_pledged,usd_type
0,202505,185341,A Year of Sanderson: Enjoy books and swag boxe...,47.0,Fiction,18,Publishing,http://www.kickstarter.com/discover/categories...,41754153.0,US,...,1.648768e+09,https://www.kickstarter.com/projects/dragonste...,True,True,successful,1648767600,1.0,1.0,41754153.24,domestic
1,202505,78471,Color e-paper smartwatch with up to 7 days of ...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,20338986.0,US,...,1.427590e+09,https://www.kickstarter.com/projects/getpebble...,True,True,successful,1427508015,1.0,1.0,20338986.27,domestic
2,202505,55106,Beginning with The Stormlight Archive and expa...,34.0,Tabletop Games,12,Games,http://www.kickstarter.com/discover/categories...,15149874.0,US,...,1.726001e+09,https://www.kickstarter.com/projects/brotherwi...,True,True,successful,1724986800,1.0,1.0,15149874.00,domestic
3,202505,62642,The COOLEST is a portable party disguised as a...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,13285226.0,US,...,1.428101e+09,https://www.kickstarter.com/projects/ryangrepp...,True,False,successful,1409360410,1.0,1.0,13285226.36,domestic
4,202505,83193,Euro-inspired dungeon crawling sequel to the 2...,34.0,Tabletop Games,12,Games,http://www.kickstarter.com/discover/categories...,12969608.0,US,...,1.588367e+09,https://www.kickstarter.com/projects/frosthave...,True,True,successful,1588366800,1.0,1.0,12969608.00,domestic


In [33]:
df = add_fx_mean(df)

In [34]:
df = df[df["currency"]!= "USD"]
df.head(5)


,collected_at,backers_count,blurb,category_id,category_name,category_parent_id,category_parent_name,category_url,converted_pledged_amount,country,...,staff_pick,state,state_changed_at,static_usd_rate,usd_exchange_rate,usd_pledged,usd_type,launched_date,fx_date,fx_daily_mean
435023,202505,12375,UNOBRUSHTM is the world’s smartest toothbrush ...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,1105454.0,DK,...,False,successful,1542906019,0.154326,0.152685,1.117333e+06,domestic,2018-10-22,2018-10-22,0.15411
435024,202505,19909,"A sustainable alternative to the cotton swabs,...",28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,718818.0,DK,...,False,successful,1558021953,0.151544,0.150091,7.257768e+05,domestic,2019-04-16,2019-04-16,0.15136
435025,202505,3299,A CO2 monitor designed to improve the indoor a...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,458167.0,DK,...,True,successful,1651759112,0.148530,0.142687,4.769279e+05,domestic,2022-04-05,2022-04-05,0.14715
435026,202505,4634,Great quality ski and snowboard goggles from C...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,465045.0,DK,...,True,successful,1543075086,0.154021,0.151925,4.714604e+05,international,2018-10-10,2018-10-10,0.15436
435027,202505,553,"Watches made from recycled Mustangs, powered b...",28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,465131.0,DK,...,True,successful,1451948340,0.143288,0.145655,4.575713e+05,domestic,2015-11-20,2015-11-20,0.14330


In [35]:
df[df["fx_daily_mean"].isna()]
df

,collected_at,backers_count,blurb,category_id,category_name,category_parent_id,category_parent_name,category_url,converted_pledged_amount,country,...,staff_pick,state,state_changed_at,static_usd_rate,usd_exchange_rate,usd_pledged,usd_type,launched_date,fx_date,fx_daily_mean
435023,202505,12375,UNOBRUSHTM is the world’s smartest toothbrush ...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,1105454.0,DK,...,False,successful,1542906019,0.154326,0.152685,1.117333e+06,domestic,2018-10-22,2018-10-22,0.15411
435024,202505,19909,"A sustainable alternative to the cotton swabs,...",28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,718818.0,DK,...,False,successful,1558021953,0.151544,0.150091,7.257768e+05,domestic,2019-04-16,2019-04-16,0.15136
435025,202505,3299,A CO2 monitor designed to improve the indoor a...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,458167.0,DK,...,True,successful,1651759112,0.148530,0.142687,4.769279e+05,domestic,2022-04-05,2022-04-05,0.14715
435026,202505,4634,Great quality ski and snowboard goggles from C...,28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,465045.0,DK,...,True,successful,1543075086,0.154021,0.151925,4.714604e+05,international,2018-10-10,2018-10-10,0.15436
435027,202505,553,"Watches made from recycled Mustangs, powered b...",28.0,Product Design,7,Design,http://www.kickstarter.com/discover/categories...,465131.0,DK,...,True,successful,1451948340,0.143288,0.145655,4.575713e+05,domestic,2015-11-20,2015-11-20,0.14330
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629142,202504,0,Let us bring Spirit War to the global audience,34.0,Tabletop Games,12,Games,http://www.kickstarter.com/discover/categories...,NaN,SG,...,False,submitted,1704701583,0.000000,NaN,NaN,domestic,1970-01-01,1970-01-01,NaN
629143,202504,0,A cooperative board game for 1-4 players from ...,34.0,Tabletop Games,12,Games,http://www.kickstarter.com/discover/categories...,NaN,SG,...,False,submitted,1609738886,0.000000,NaN,NaN,domestic,1970-01-01,1970-01-01,NaN
629144,202504,0,"Enter ARMONS, your AR Partners that you can nu...",272.0,Mobile Games,12,Games,http://www.kickstarter.com/discover/categories...,NaN,SG,...,False,submitted,1647189291,0.000000,NaN,NaN,domestic,1970-01-01,1970-01-01,NaN
629145,202504,0,"Chain-K not just stores your memory, but can b...",NaN,NaN,16,Technology,http://www.kickstarter.com/discover/categories...,NaN,SG,...,False,submitted,1610536265,0.000000,NaN,NaN,domestic,1970-01-01,1970-01-01,NaN


In [36]:
df["usd_goal_fx"] = (df["goal"] * df["fx_daily_mean"]).round(0)

In [37]:
df = df[["country", "currency","goal", "fx_daily_mean", "usd_goal_fx"]]
df

,country,currency,goal,fx_daily_mean,usd_goal_fx
435023,DK,DKK,100223.0,0.15411,15445.0
435024,DK,DKK,89000.0,0.15136,13471.0
435025,DK,DKK,50000.0,0.14715,7358.0
435026,DK,DKK,50000.0,0.15436,7718.0
435027,DK,DKK,1400000.0,0.14330,200620.0
...,...,...,...,...,...
629142,SG,SGD,0.0,NaN,NaN
629143,SG,SGD,0.0,NaN,NaN
629144,SG,SGD,0.0,NaN,NaN
629145,SG,SGD,0.0,NaN,NaN
